In [3]:
# 맨 마지막 줄 함수에 (시작년, 끝년)을 넣고
# 본인 cid, secret 입력
# 모두실행
import billboard
from pprint import pprint
from datetime import datetime, timedelta
import numpy as np
import json
curdate = datetime.now()

yearchartarr = []
def getYearsChart(s_year, e_year):
    curday = datetime(s_year, 1, 1)
    print(curday.year <= e_year)
    while (curday.year <= e_year):
        dayformat = curday.strftime('%Y-%m-%d')
        charts = billboard.ChartData('hot-100', date=dayformat)
        curday += timedelta(weeks=1)
        dayarr = []
        for song in charts:
            arr = [dayformat, song.title, song.artist, song.rank, []]
            dayarr.append(arr)
        yearchartarr.append(dayarr)
        print(len(yearchartarr))

# json으로 저장
def getBillboardData(s_year, e_year):
    getYearsChart(s_year, e_year)
    json_data = json.dumps(yearchartarr)
    pprint(json_data)
    with open(f'data/billboard/billboard_{s_year}to{e_year}.json', 'w') as f:
        f.write(json_data)

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd
import json
import time
import re
import difflib
from sklearn.preprocessing import LabelEncoder


cid = "33b399df79d5425381e6e79793d2c110" 
secret = "b1c455604b2149ec990eb8fafbf311d0"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

af = ['danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
columns = ['date', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
trackIDs = []
songdict = {}

# get json file of billboard

def getJSONFile(s_year, e_year):
    with open(f'data/billboard/billboard_{s_year}to{e_year}.json', 'r') as f:
        data = f.read()
        data = json.loads(data)
    return data

def getSongSet(data):
    songSet = set()
    for chart in data:
        for song in chart:
            songSet.add((song[1], song[2]))
    return songSet

def getIDsAt(num, ls):
    temp = []
    for i in range(num * 77, num * 77 + 77):
        if i < len(ls) - 1:
            # artist = ls[i][1].replace(" x ", " & ").replace(" X ", " & ").replace("Featuring", "&").replace("featuring", "&").replace("With", "&").replace("with", "&").replace("And", "&").replace("and", "&")
            # raw string으로 변환 처리
            artist = re.sub(r"( x | X |Featuring|featuring|With|with|And|and)", "&", ls[i][1], flags=re.IGNORECASE)
            search = sp.search(f"{ls[i][0]} - {artist}", limit=1, type=['track', 'artist'])
            if search.get('tracks') == None:
                continue
            try:
                track = search['tracks']['items'][0]
                trackID = track['id']
                trackName = track['name']
                name = ls[i][0]
                # trackName과 name의 문자열을 비교해서 같은 정도가 어느정도인지 확인
                similarity = difflib.SequenceMatcher(None, trackName.lower(), name.lower()).ratio()
                if (trackName.lower() in name.lower()) or (name.lower() in trackName.lower()) or (similarity >= 0.5):
                    temp.append(trackID)
                    songdict[trackID] = [f"{ls[i][0]} - {ls[i][1]}", []]
            except:
                pprint(search)
        time.sleep(2)
    trackIDs.append(temp)
    
def getIDs(len, ls):
    for i in range(len):
        getIDsAt(i, ls)
        
def getFeaturesAt(num):
    features = sp.audio_features(tracks=trackIDs[num])
    for id, feature in zip(trackIDs[num], features):
        try:
            songdict[id] = [songdict[id][0], [feature["danceability"], feature["energy"], feature["loudness"], feature["mode"], feature["speechiness"], feature["acousticness"], feature["instrumentalness"], feature["liveness"], feature["valence"], feature["tempo"]]]
        except:
            songdict[id] = [songdict[id][0], None]
            
def getFeatures(len):
    for i in range(len):
        getFeaturesAt(i)
        time.sleep(5)
        
def getSpotifyData(s_year, e_year):
    data = getJSONFile(s_year, e_year)
    songSet = getSongSet(data)
    ls = list(songSet)
    length = len(ls) // 77 + 1
    
    getIDs(length, ls)
    getFeatures(len(trackIDs))
    featuresDict = {}
    for key, value in songdict.items():
        featuresDict[value[0]] = value[1]
        
    for chart in data:
        for song in chart:
            query = song[1] + " - " + song[2]
            song[4] = featuresDict.get(query)
    
    label = []
    for chart in data:
        for song in chart:
            if song[4] == None:
                label.append(50)
            else:
                label.append(song[3])
    label = np.array(label)
    dataList = []
    index = []
    arr = []
    for chart in data:
        for song in chart:
            index.append(song[0])
            if song[4] != None:
                arr = song[4]
                
            else:
                arr = np.empty(10)
                arr[:] = np.nan
            # arr 앞에 song[0]을 추가
            arr = np.concatenate(([song[0]], arr))
            dataList.append(arr)
    df = pd.DataFrame(data=dataList, index=index, columns=columns)
    label = pd.Series(label, index=index)

    for col in columns:
        if (df[col].dtype == "object") or (df[col].dtype == "O"):
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
    df.fillna(df.median(), inplace=True)
    
    df.to_csv(f"data/dataframes/dataframe_{s_year}to{e_year}.csv")
    label.to_csv(f"data/dataframes/label_{s_year}to{e_year}.csv")


In [5]:
getBillboardData(2023, 2023)

True
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
('[[["2023-01-01", "All I Want For Christmas Is You", "Mariah Carey", 1, []], '
 '["2023-01-01", "Rockin\' Around The Christmas Tree", "Brenda Lee", 2, []], '
 '["2023-01-01", "Jingle Bell Rock", "Bobby Helms", 3, []], ["2023-01-01", '
 '"Last Christmas", "Wham!", 4, []], ["2023-01-01", "A Holly Jolly Christmas", '
 '"Burl Ives", 5, []], ["2023-01-01", "It\'s The Most Wonderful Time Of The '
 'Year", "Andy Williams", 6, []], ["2023-01-01", "Feliz Navidad", "Jose '
 'Feliciano", 7, []], ["2023-01-01", "Anti-Hero", "Taylor Swift", 8, []], '
 '["2023-01-01", "The Christmas Song (Merry Christmas To You)", "Nat '
 '\\"King\\" Cole", 9, []], ["2023-01-01", "Unholy", "Sam Smith & Kim Petras", '
 '10, []], ["2023-01-01", "Kill Bill", "SZA", 11, []], ["2023-01-01", "Santa '
 'Tell Me", "Ariana Grande", 12, []], ["2023-01-01", "Sleigh Ride", "

In [6]:
    
getSpotifyData(2023, 2023)

: 